# 1. 도서 제목 및 도서 설명 정의

In [ ]:
doc5_titles = [
    "자바스크립트언어",
    "일본관광시기",
    "파이썬언어",
    "기계학습기초",
    "스페인방문계절"
]

doc5_desc = [
    "자바스크립트는 웹 개발에 필수적인 프로그래밍 언어입니다.",
    "일본은 벚꽃이 피는 봄이 관광하기 가장 좋은 시기입니다.",
    "파이썬 언어는 데이터분석과 기계학습에 효율적인 프로그래밍 언어입니다",
    "기계학습은 데이터를 활용하여 컴퓨터가 학습하도록 하는 기술입니다.",
    "스페인은 날씨가 온화한 봄이나 가을에 방문하는 것이 이상적입니다."
]

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="UPSATGE_API_KEY",
    base_url="https://api.upstage.ai/v1/solar"
)

response = client.embeddings.create(
    model = "embedding-passage",
    input = doc5_desc
).data

embedding_vectors = [i.embedding for i in response]

In [ ]:
# 임베딩 벡터 데이터만 리스트로 추출
embedding_vectors = [i.embedding for i in response]
# 생성된 벡터 데이터수 == 문서 수
print(len(embedding_vectors))
# 업스테이지 임베딩 모델의 벡터 차원 == 4096
print(len(embedding_vectors[0]))

# 2. 문서 간 코싸인 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(embedding_vectors, embedding_vectors)
print(similarities)

# 3. 첫번째 책의 유사도 정렬

In [ ]:
# 첫번째 책의 각 책과의 유사도 정보 (index, 유사도)
sim_scores = list(enumerate(similarities[0]))

# 유사 기준 내림차순 정렬
sim_scores.sort(key = lambda x: x[1], reverse = True)

# 자기 자신은 제외한 상위 3개 문서 출력
print(sim_scores[1:4] )

# 4. 책 정보가 유사한 도서 추천

In [ ]:
books_titles = doc5_titles

def recommendations(title):
    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    if title in books_titles :
        idx=books_titles.index(title)
        similar_doc = similarities[idx]
    else :
        print("도서 정보가 존재하지 않습니다.")
        return []

    # 입력된 책과 줄거리(document embedding)가 유사한 책 3개 선정.
    sim_scores = list(enumerate(similar_doc)) # (index, 유사도) 튜플의 리스트
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True) # 유사도 높은 순서로 sorting
    sim_scores = sim_scores[1:4] # 자기 자신은 제외한 상위 3개
    print(sim_scores)

    similar_books_titles = []
    # 유사한 책 제목 출력
    for index, book_info in sim_scores:
        title=books_titles[index]
        similar_books_titles.append(title)

    return similar_books_titles

In [ ]:

title="소년이 온다"
#title = "이반 일리치의 죽음"
recommendations(title)

In [ ]:
title="자바스크립트언어"
# title="스페인방문계절"
recommendations(title)

In [ ]:
doc5_titles = [
    "일본관광시기",
    "자바스크립트언어",
    "파이썬언어",
    "기계학습기초",
    "스페인방문계절"
]

doc5_desc = [
    "일본은 벚꽃이 피는 봄이 관광하기 가장 좋은 시기입니다.",
    "자바스크립트는 웹 개발에 필수적인 프로그래밍 언어입니다.",
    "파이썬 언어는 데이터분석과 기계학습에 효율적인 프로그래밍 언어입니다",
    "기계학습은 데이터를 활용하여 컴퓨터가 학습하도록 하는 기술입니다.",
    "스페인은 날씨가 온화한 봄이나 가을에 방문하는 것이 이상적입니다."
]

# 5. [참고1] 임베딩 벡터 파일, 코싸인 유사도 파일 저장하고 불러쓰기

In [ ]:
import pickle

# pickle로 저장하기
with open("embedding_vectors_upstage_doc5.pickle","wb") as f:
    pickle.dump(embedding_vectors, f)

# pickle로 불러오기
with open("embedding_vectors_upstage_doc5.pickle","rb") as f:
	embedding_vectors=pickle.load(f)

# pickle로 저장하기
with open("similarities_upstage_doc5.pickle","wb") as f:
    pickle.dump(similarities, f)

# pickle로 불러오기
with open("similarities_upstage_doc5.pickle","rb") as f:
	similarities=pickle.load(f)

# 6. [참고2] 임베딩 벡터 시각화
https://projector.tensorflow.org/



In [ ]:
import os
import tensorflow as tf
import pandas as pd

LOG_DIR='./log_dir'
# 경로에 해당하는 directory가 없으면 생성
os.makedirs(LOG_DIR, exist_ok=True)

weights = tf.Variable(embedding_vectors)
df = pd.DataFrame(weights)
df.to_csv(os.path.join(LOG_DIR, "embedding_vectors_upstage_doc5.tsv"), sep = '\t', index = False, header=False)

In [ ]:
with open(os.path.join(LOG_DIR, 'metadata_upstage_doc5.tsv'), "w", encoding='utf-8') as f:
    for title in doc5_titles :
        f.write(title+'\n')